
TLDR; How to use fuse.js to implement a simple search?

# How to interpret questions in jupyter notebooks?

How to turn jupyter notebooks in to modules?

How to answer questions using jupyter notebooks and simple search?

How to search jupter notebooks for questions and answers?



In [3]:
// install fuse
try {
    require.resolve('fuse.js');
} catch (e) {
    require('child_process').execSync('npm install fuse.js');
}

if (typeof cellCache == 'undefined') {var cellCache = []};

var Fuse = require('fuse.js');
var fuse = new Fuse(cellCache, {
    shouldSort: true, 
    keys: ['question'],
    id: 'id'
});

var interpret = (message) => fuse.search(message);
(interpret);



[Function: interpret]

How to cache notebook questions in fuse?

In [4]:
var corePath = path.resolve(__dirname);
var getCells = require(corePath).getCells;
var cacheCells = (cells, notebook) => {}; // defined below
// interpret notebooks by importing them in to cache
(interpretNotebook = (notebook) => {
    return getCells(notebook, ['*', 'markdown', 'code'])
        .then(cells => cacheCells(cells, notebook));
});



[Function: interpretNotebook]

How to find questions leading up to jupter cells?

How to read markdown leading up to code cells?


In [5]:
// TODO: use the m flag option for regexp?
var re = new RegExp('(^|\\n).*?\\?(\\s+|$)', 'ig');
var re2 = new RegExp('(^|\\n)//.*\\?(\\s+|$)', 'ig')

var accumulateMarkdown = (cells) => {
    var counter = 0, prev = [];
    return cells.reduce((md, c) => {
        counter++;
        var source = c.source.join('');
        if (c.cell_type == 'markdown') {
            prev.push(source);
            return md;
        } else if(c.cell_type !== 'code') {
            return md;
        }
        // TODO: improve the counter
        var cell = {code: source, markdown: prev, from: counter-1, to: counter};
        prev = [];
        md.push(cell);
        return md;
    }, []);
};

// How to convert a string to an Array of RegEx matches
var regexToArray = (ex, str, i = 0) => {
    var co = []; var m;
    while ((m = re.exec(str)) && co.push(m[i]));
    return co;
};
(regexToArray);

[Function: regexToArray]



This is a complex question to answer.  Perhaps using IBM Watson language alchemy?

How to find functions in jupyter notebooks?



In [6]:

// TODO: interpret markdown leading up to code results and find the resulting function in the list
// for now, the boring solution is to assume all markdown output is a question?

// How to store code markdown results for later use?
if (typeof cellCache == 'undefined') { var cellCache = []; }
if (typeof cacheIds == 'undefined') { var cacheIds = {}; }
(cacheCells = ((cells, notebook) => {
    var filename = path.basename(notebook);
    var newCache = accumulateMarkdown(cells);
    newCache.forEach((c, i) => {
        if(typeof cacheIds[filename+'['+i+']'] !== 'undefined') {
            return;
        }
        var questions = regexToArray(re, c.markdown)
            .concat(regexToArray(re, c.source)
                    .map(r => r.replace(/how to|?/ig, '')));
        cacheIds[filename+'['+i+']'] = Object.assign({}, c, {
            id: filename+'['+i+']',
            filename: notebook,
            questions: questions,
            notebook: filename,
            function: global[filename+'['+i+']']
        });
        questions.forEach((q) => cellCache.push({
            id: filename+'['+i+']',
            question: q
        }));
    });
    return cellCache;
}));



[Function: cacheCells]


How to test caching works?


In [7]:

var testNotebook = 'test import.ipynb';
var testCells = [
    {cell_type: 'code', source: []},
    {cell_type: 'markdown', source: ["how to test for errors line 2?"]},
    {cell_type: 'code', source: ["(function (err) { throw error; })"]}
];
global[testNotebook+'[1]'] 
    = eval(testCells[testCells.length-1].source.join('').trim());
var markdownCache = cacheCells(testCells, testNotebook);
if(markdownCache[0].id == testNotebook+'[1]') {
    $$.sendResult(cacheCells);
}


[Function: cacheCells]

How to interpret a jupyter {directory}?

How to display interpreted results in markdown?

How to test the interpreter works?


In [8]:
// How to represent search results in markdown?
var resultMarkdown = (res) => {
    return ('\n\n\n' + res.length + ' match' 
   + (res.length !== 1 ? 'es' : '')
   + ' found: ' + res.join(' , ') + '\n\n\n'
   + (res.length > 0 
      ? ('\n\n\n' + cacheIds[res[0]].markdown.join('\n') + '\n\n\n' 
        + '```\n\n\n' + cacheIds[res[0]].code + '\n\n\n```\n\n\n') 
      : ''));
};

var interpretMarkdown = (results) => 
    (typeof results[0] !== 'undefined' && typeof results[0] !== 'string'
    ? results.reduce((str, res) => str += resultMarkdown(res), '')
    : resultMarkdown(results));

(interpretMarkdown);


[Function: interpretMarkdown]

In [11]:
var corePath = path.resolve(__dirname);
var getCells = require(corePath).getCells;

var getFresher = (cache) => {
    return getCells(cache.filename, ['*', 'markdown', 'code'])
        .then(r => {
            cache.fresher = accumulateMarkdown(r.slice(cache.from, cache.to))[0].code;
            return cache;
        });
};

var interpretObject = (results) =>
    (typeof results[0] !== 'undefined' && typeof results[0] !== 'string'
    ? Promise.all(results.map((res) => Promise.all(res.map(r => {
        delete cacheIds[r].function;
        return getFresher(cacheIds[r]);
    }))))
    : Promise.all(results.map(r => {
        delete cacheIds[r].function;
        return getFresher(cacheIds[r]);
    })));
$$.sendResult(interpretObject);


[Function: interpretObject]

How to execute a cell from interpreted search results?


In [10]:
var importer = require(__dirname);
var runAllPromises = importer.runAllPromises;
var imported = importer.imported;
// An intention is the same as interpret but gracefully executes or returns markdown
(intend = (message, a1, a2, a3, a4, a5, a6, a7, a8, a9) => {
    var intentions = message;
    if (typeof message === 'string') {
        intentions = [message];
    }
    return runAllPromises(intentions.map(m => {
        return Promise.resolve(interpret(m))
            .then(r => {
            if(r.length === 1 || 
                typeof imported[cacheIds[r[0]].notebook][r[0]] 
                    !== 'undefined') {
                // TODO: dependency injection here?
                return imported[cacheIds[r[0]].notebook][r[0]]
                    .apply(this, [a1, a2, a3, a4, a5, a6, a7, a8, a9]);
            } else {
                return interpretMarkdown([r]);
            }
        })
    }));
});



[Function: intend]